In [1]:
# import necessary libraries
# virtual environment is tied to python 3.10.2
import math
import numpy as np
import sympy as sm
sm.init_printing(use_latex='mathjax')
import matplotlib.pyplot as plt
import matplotlib.patches as pch
%matplotlib inline
from matplotlib.ticker import MaxNLocator # to make integer ticks on x axis

In [2]:
# define variables
t               = sm.symbols("t", real=True)        # variable of integration
n               = sm.symbols("n", integer=True)     # n-th harmonic
T               = sm.symbols("T", positive=True)    # period
A               = sm.symbols("A", real=True)        # amplitude

In [3]:
# define target function
# f(t) = A*(1-(t/tau)) if 0 <= t < tau and 0 if tau <= t < (2*tau)
# T = 2*tau since duty cycle is set to 50%
T_val   = 1e-6      # Period of sawtooth signal
tau_val = T_val/2
N       = 10        # Generate Fourier Series to 10th harmonic
A_val   = 1

ft = (A * (1 - t / tau_val))

In [4]:
# define target function as a fourier series approximation
def fourier_series(ft, n, T, t_eval):

    omega = 2 * sm.pi  / T

    a_0 = 2/T * sm.integrate(ft, (t,0,(T/2)))
    a_n = 2/T * sm.integrate(ft*sm.cos(omega*n*t), (t,0,(T/2)))
    b_n = 2/T * sm.integrate(ft*sm.sin(omega*n*t), (t,0,(T/2)))

    # Construct the Fourier series expression
    fourier_series = a_0 / 2
    for i in range(1, n + 1):
        fourier_series += a_n.subs(n, i) * sm.cos(i * omega * t) + b_n.subs(n, i) * sm.sin(i * omega * t)

    # Numerically evaluate the Fourier series for plotting
    fourier_series_values = np.array([fourier_series.subs(t, ti) for ti in t_eval], dtype=np.float64)
    
    return fourier_series, fourier_series_values

In [5]:
def plot_results(t_vals, fourier_series_values, n, T):
    """Plots Fourier series approximation of a function."""
    plt.figure(figsize=(10, 6))
    plt.plot(t_vals, fourier_series_values, label=f'Fourier Series ({n} Harmonics)')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title(f'Half-Sawtooth Wave and its Fourier Series Approximation (T={T}s)')
    plt.legend()
    plt.grid(True)
    plt.show()

num_points = 1000 # Number of points for plotting
sim_time = np.linspace(0, 2 * T_val, num_points)  # Time values for two periods

# Calculate the Fourier series
unevaluated_fourier_series, fourier_series_values = fourier_series(A, T, N, sim_time)

# Plot the results
plot_results(sim_time, np.real(fourier_series_values), N, T_val)

TypeError: 'Add' object cannot be interpreted as an integer

In [ ]:
from sympy import symbols, pi, cos, sin, integrate, Sum
from sympy.plotting import plot
import matplotlib.pyplot as plt
import numpy as np

def half_sawtooth_fourier_series(A, T, num_harmonics, t_vals):
    """
    Calculates the Fourier series of a half-sawtooth wave and compares
    it with the analytical expression.

    Args:
        A (float): Amplitude of the sawtooth wave.
        T (float): Period of the sawtooth wave.
        num_harmonics (int): Number of harmonics to calculate.
        t_vals (list/numpy array): Time values for plotting.

    Returns:
        tuple: (analytical_fourier, numerical_fourier)
               analytical_fourier: Symbolic expression of the Fourier series.
               numerical_fourier:  Numerical values of the Fourier series
    """
    t = symbols('t')
    n = symbols('n')
    omega = 2 * pi / T
    tau = T / 2  # 50% duty cycle

    # Calculate Fourier coefficients analytically using SymPy
    a0 = (2 / T) * integrate(A * (1 - t / tau), (t, 0, tau))
    an = (2 / T) * integrate(A * (1 - t / tau) * cos(n * omega * t), (t, 0, tau))
    bn = (2 / T) * integrate(A * (1 - t / tau) * sin(n * omega * t), (t, 0, tau))

    # Construct the Fourier series expression
    fourier_series = a0 / 2
    for i in range(1, num_harmonics + 1):
        fourier_series += an.subs(n, i) * cos(i * omega * t) + bn.subs(n, i) * sin(i * omega * t)

    # Numerically evaluate the Fourier series for plotting
    fourier_series_values = np.array([fourier_series.subs(t, ti) for ti in t_vals], dtype=float)

    return fourier_series, fourier_series_values

def plot_results(t_vals, original_signal, fourier_series_values, num_harmonics, A, T):
    """Plots the original signal and its Fourier series approximation."""
    plt.figure(figsize=(10, 6))
    plt.plot(t_vals, original_signal, label='Original Signal', color='blue')
    plt.plot(t_vals, fourier_series_values, label=f'Fourier Series ({num_harmonics} Harmonics)', color='red')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude (V)')
    plt.title(f'Half-Sawtooth Wave and its Fourier Series Approximation (A={A}V, T={T}s)')
    plt.legend()
    plt.grid(True)
    plt.show()

# Parameters
A = 1.0  # Amplitude (V)
T = 1 / 10e6  # Period (s), 10 MHz frequency
num_harmonics = 10  # Number of harmonics to calculate
num_points = 1000 # Number of points for plotting
t_vals = np.linspace(0, 2 * T, num_points)  # Time values for two periods

# Generate the original half-sawtooth signal
original_signal = np.array([A * (1 - (t % T) / (T / 2)) if (t % T) < (T / 2) else 0 for t in t_vals])

# Calculate the Fourier series
analytical_fourier, numerical_fourier = half_sawtooth_fourier_series(A, T, num_harmonics, t_vals)

print("Analytical Fourier Series Expression:")
print(analytical_fourier)

# Plot the results
plot_results(t_vals, original_signal, numerical_fourier, num_harmonics, A, T)
